In [56]:
import random
import chess
import chess.svg
from IPython.display import SVG, display

In [57]:
class Node:
    def __init__(self, board, parent):
        self.board = board
        self.parent = parent
        self.value = 0
        self.count = 0
        self.explored = []
        self.potential = list(map(str, list(self.board.legal_moves)))
    
    def expand(self):
        if len(self.potential) > 0:
            newBoard = self.board.copy()
            newBoard.push_uci(self.potential.pop())
            newNode = Node(newBoard, self)
            self.explored.append(newNode)
            newNode.simulateAndBackPropagate()
        else:
            if self.board.turn:                
                maxVal = -1_000_000
                bestSucc = None
                for i in range(len(self.explored)):
                    value = self.explored[i].value / self.explored[i].count
                    if value > maxVal:
                        maxVal = value
                        bestSucc = self.explored[i]
            else:
                minVal = 1_000_000
                bestSucc = None
                for i in range(len(self.explored)):
                    value = self.explored[i].value / self.explored[i].count
                    if value < minVal:
                        minVal = value
                        bestSucc = self.explored[i]
            bestSucc.simulateAndBackPropagate()
    
    def simulateAndBackPropagate(self):
        tempBoard = self.board.copy()
        value = 0
        while not tempBoard.is_game_over():
            move = random.choice(list(map(str, list(tempBoard.legal_moves))))
            tempBoard.push_uci(move)
        if tempBoard.outcome():
            if tempBoard.outcome().result() == '1-0':
                value = 1
            elif tempBoard.outcome().result() == '0-1':
                value = -1
        
        self.count += 1
        self.value += value
        current = self
        while current.parent:
            current.parent.count += 1
            current.parent.value += value
            current = current.parent
    
    def getBestMove(self):
        bestMove = None
        if self.board.turn:                
            maxVal = -1_000_000
            bestMove = None
            for i in range(len(self.explored)):
                value = self.explored[i].value / self.explored[i].count
                if value > maxVal:
                    maxVal = value
                    bestMove = self.explored[i].board.move_stack[-1]
        else:
            minVal = 1_000_000
            bestMove = None
            for i in range(len(self.explored)):
                value = self.explored[i].value / self.explored[i].count
                if value < minVal:
                    minVal = value
                    bestMove = self.explored[i].board.move_stack[-1]
        return str(bestMove)

In [ ]:
numberOfMoves = 4
iterations = 100
board = chess.Board()
for _ in range(4):
    node = Node(board, None)
    for i in range(iterations):
        node.expand()
    move = node.getBestMove()
    board.push_uci(move)
    #display(SVG(chess.svg.board(board, size=400)))